# EEG Viewer
![status](https://img.shields.io/badge/status-in%20progress-orange)



<div style="text-align: center;">
    <img src="./assets/230524_eeg-viewer.png" alt="eeg viewer preview" width="450"/>
</div>

## Summary

This workflow is intended to demonstrate the visualization of a set of 1D EEG timeseries with HoloViz and Bokeh tools.

For details specific to this workflow, such as goals, specifications, and bottlenecks, please see this workflow's [readme](./readme_eeg-viewer.md).

For a summary of EEG research, data, and software, please see [neuro/wiki/EEG-notes](https://github.com/holoviz-topics/neuro/wiki/EEG-notes).

## Imports and config

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Requirements</p>
    <p>This workflow notebook requires the <a href="./environment.yml">environment</a> specified in this workflow directory.</p>
</div>


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from scipy.stats import zscore

# HoloViz and Bokeh
import colorcet as cc
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import Dataset
from bokeh.models import HoverTool, WheelZoomTool
import panel as pn; pn.extension(template='material')

# Neuro repo
from neurodatagen.eeg import generate_eeg_powerlaw
from hvneuro import download_file

## Synthetic data pipeline

The `generate_eeg_powerlaw` function synthesizes EEG data as high-pass filtered pink noise power law time series by default. The function returns a 2D numpy array of synthetic EEG data (in microvolts) shaped as (number of channels, total samples), a 1D time array (in seconds), and a list of channel names. Parameters such as the high-pass filter factor (in Hz) and an amplitude scaling factor allow customization of the generated data.

### Generate synthetic data

In [ ]:
%%time
n_channels = 25
n_seconds = 30
fs = 512

data, time, channels = generate_eeg_powerlaw(n_channels, n_seconds, fs, channel_prefix='EEG')

print(f'shape: {data.shape} (n_channels, samples) ')
data

### Visualize synthetic data

In [ ]:
# max_ch_disp = 15  # max channels to initially display
# max_t_disp = 2 # max time in seconds to initially display
# spacing = 5.5  # Spacing between channels
# clim_mul = 1.2 # color range multiplier for minimap. lower will saturate more.

# # Annotations
# annotation = hv.VSpan(1, 1.5) # example annotation (start, end) time

# # Create a hv.Curve element per chan
# channel_curves = []
# max_data = data.max()
 
# hover = HoverTool(tooltips=[
#     ("Channel", "@channel"),
#     ("Time", "$x s"),
#     ("Amplitude", "@original_amplitude µV")])

# offset = np.std(data) * spacing
# for i, channel_data in enumerate(data):
#     offset_data = channel_data + (i * offset)
#     max_data = max(offset_data.max(), max_data) # update max
#     ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
#     channel_curves.append(
#         hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
#             color="black", line_width=1,
#             tools=[hover, 'xwheel_zoom'], shared_axes=False))

# yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# # set maintain focus to False to allow independence for zoom out against a single hardbound
# # def set_maintain_focus(plot, element):
# #     wheel_zoom = plot.state.select(type=WheelZoomTool)
# #     if wheel_zoom:
# #         wheel_zoom[0].maintain_focus = False
        
# # Create an overlay of curves
# eeg_viewer = (annotation * hv.Overlay(channel_curves, kdims="Channel")).opts(
#     padding=0, xlabel="Time (s)", ylabel="Channel", #default_tools=['hover', 'pan', 'box_zoom', 'save', 'reset'],
#     yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
#     shared_axes=False, backend_opts={
#         "x_range.bounds": (time.min(), time.max()),
#         "y_range.bounds": (data.min(), max_data)})

# # Minimap
# y_positions, _ = zip(*yticks) # use positions of yticks for yaxis of minimap image
# z_data = zscore(data, axis=1)
# minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
# minimap = minimap.opts(
#     cmap="RdBu_r", colorbar=False, xlabel='', alpha=.8, yticks=[yticks[0], yticks[-1]],
#     height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# # Create RangeToolLink between the minimap and the main EEG viewer 
# # (quirk: apply to just one eeg trace and it will apply to all. see HoloViews #4472)
# max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
# RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
#               boundsx=(None, max_t_disp),
#               boundsy=(None, max_y_disp))

# eeg_app = pn.Column((eeg_viewer + minimap * annotation).cols(1), min_height=650)#.servable(target='main', title='EEG Viewer with HoloViz and Bokeh')
# eeg_app

# No subcoords, testing with simple data

In [ ]:
# import numpy as np
# import holoviews as hv
# from bokeh.models import HoverTool
# from holoviews.plotting.links import RangeToolLink
# from scipy.stats import zscore
# from holoviews.operation.datashader import rasterize

# hv.extension('bokeh')


# N_CHANNELS = 10
# N_SECONDS = 5
# SAMPLING_RATE = 200
# INIT_FREQ = 2  # Initial frequency in Hz
# FREQ_INC = 5  # Frequency increment
# AMPLITUDE = 1

# # Generate time and channel labels
# total_samples = N_SECONDS * SAMPLING_RATE
# time = np.linspace(0, N_SECONDS, total_samples)
# channels = [f'EEG {i}' for i in range(N_CHANNELS)]

# # Generate sine wave data
# data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
#                      for i in range(N_CHANNELS)])

# max_ch_disp = 5  # max channels to initially display
# max_t_disp = 2 # max time in seconds to initially display
# spacing = 3.5  # Spacing between channels
# clim_mul = 1.2 # color range multiplier for minimap. lower will saturate more.

# # Annotations
# annotation = hv.VSpan(1, 1.5) # example annotation (start, end) time

# # Create a hv.Curve element per chan
# hover = HoverTool(tooltips=[
#     ("Channel", "@channel"),
#     ("Time", "$x s"),
#     ("Amplitude", "@original_amplitude µV")])

# # set offset for data (non-subcoord approach)
# offset = np.std(data) * spacing
# offset_data = data + (np.arange(len(data))[:, np.newaxis] * offset)
# max_data = offset_data.max()

# y_positions = np.arange(len(channels)) * offset
# yticks = list(zip(y_positions, channels))

# channel_curves = []
# for i, channel_data in enumerate(data):
#     ds = hv.Dataset((time, offset_data[i,:], channel_data, channels[i]), 
#                  ["Time", "Amplitude", "original_amplitude", "channel"])
#     channel_curves.append(
#         hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
#             color="black", line_width=1,
#             tools=[hover, 'xwheel_zoom'], shared_axes=False))
        
# eeg_viewer = (annotation * hv.Overlay(channel_curves, kdims="Channel")).opts(
#     padding=0, xlabel="Time (s)", ylabel="Channel",
#     yticks=yticks, show_legend=False, aspect=3, responsive=True,
#     shared_axes=False, backend_opts={
#         "x_range.bounds": (time.min(), time.max()),
#         "y_range.bounds": (data.min(), max_data)})

# z_data = zscore(data, axis=1)
# minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
# minimap = minimap.opts(
#     cmap="RdBu_r", colorbar=False, xlabel='', alpha=.8, yticks=[yticks[0], yticks[-1]],
#     height=100, responsive=True, default_tools=[''], shared_axes=False,
#     clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# max_y_disp = np.min([N_CHANNELS+1, max_ch_disp+1])
# RangeToolLink(minimap, eeg_viewer, axes=["x", "y"],
#               boundsx=(None, max_t_disp),
#               boundsy=(None, max_y_disp))

# eeg_app = (eeg_viewer + minimap * annotation).cols(1)
# eeg_app

In [ ]:
# import numpy as np
# import holoviews as hv
# from bokeh.models import HoverTool
# from holoviews.plotting.links import RangeToolLink
# from scipy.stats import zscore
# from holoviews.operation.datashader import rasterize

# hv.extension('bokeh')


# N_CHANNELS = 10
# N_SECONDS = 5
# SAMPLING_RATE = 200
# INIT_FREQ = 2  # Initial frequency in Hz
# FREQ_INC = 5  # Frequency increment
# AMPLITUDE = 1

# # Generate time and channel labels
# total_samples = N_SECONDS * SAMPLING_RATE
# time = np.linspace(0, N_SECONDS, total_samples)
# channels = [f'EEG {i}' for i in range(N_CHANNELS)]

# # Generate sine wave data
# data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
#                      for i in range(N_CHANNELS)])

# max_ch_disp = 5  # max channels to initially display
# max_t_disp = 2 # max time in seconds to initially display
# spacing = 3.5  # Spacing between channels
# clim_mul = 1.2 # color range multiplier for minimap. lower will saturate more.

# # Annotations
# annotation = hv.VSpan(1, 1.5) # example annotation (start, end) time

# # Create a hv.Curve element per chan
# hover = HoverTool(tooltips=[
#     ("Channel", "@channel"),
#     ("Time", "$x s"),
#     ("Amplitude", "@original_amplitude µV")])

# # set offset for data (non-subcoord approach)
# offset = np.std(data) * spacing
# offset_data = data + (np.arange(len(data))[:, np.newaxis] * offset)
# max_data = offset_data.max()

# y_positions = np.arange(len(channels)) * offset
# yticks = list(zip(y_positions, channels))

# channel_curves = []
# for i, channel_data in enumerate(data):
#     ds = hv.Dataset((time, offset_data[i,:], channel_data, channels[i]), 
#                  ["Time", "Amplitude", "original_amplitude", "channel"])
#     channel_curves.append(
#         hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
#             color="black", line_width=1,
#             tools=[hover, 'xwheel_zoom'], shared_axes=False))
        
# eeg_viewer = (annotation * hv.Overlay(channel_curves, kdims="Channel")).opts(
#     padding=0, xlabel="Time (s)", ylabel="Channel",
#     yticks=yticks, show_legend=False, aspect=3, responsive=True,
#     shared_axes=False, backend_opts={
#         "x_range.bounds": (time.min(), time.max()),
#         "y_range.bounds": (data.min(), max_data)})

# z_data = zscore(data, axis=1)
# minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
# minimap = minimap.opts(
#     cmap="RdBu_r", colorbar=False, xlabel='', alpha=.8, yticks=[yticks[0], yticks[-1]],
#     height=100, responsive=True, default_tools=[''], shared_axes=False,
#     clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# max_y_disp = np.max([N_CHANNELS+1, max_ch_disp+1])
# RangeToolLink(minimap, eeg_viewer, axes=["x", "y"],
#               boundsx=(None, max_t_disp),
#               boundsy=(None, max_y_disp))

# eeg_app = (eeg_viewer + minimap * annotation).cols(1)
# eeg_app

## subcoords 

In [ ]:
# import numpy as np
# import holoviews as hv
# from bokeh.models import HoverTool
# from holoviews.plotting.links import RangeToolLink
# from scipy.stats import zscore
# from holoviews.operation.datashader import rasterize

# hv.extension('bokeh')

# N_CHANNELS = 10
# N_SECONDS = 5
# SAMPLING_RATE = 200
# INIT_FREQ = 2  # Initial frequency in Hz
# FREQ_INC = 5  # Frequency increment
# AMPLITUDE = 1

# # Generate time and channel labels
# total_samples = N_SECONDS * SAMPLING_RATE
# time = np.linspace(0, N_SECONDS, total_samples)
# channels = [f'EEG {i}' for i in range(N_CHANNELS)]

# # Generate sine wave data
# data = np.array([AMPLITUDE * np.sin(2 * np.pi * (INIT_FREQ + i * FREQ_INC) * time)
#                      for i in range(N_CHANNELS)])

# hover = HoverTool(tooltips=[
#     ("Channel", "@channel"),
#     ("Time", "$x s"),
#     ("Amplitude", "$y µV")
# ])

# channel_curves = []
# for channel, channel_data in zip(channels, data):
#     ds = hv.Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
#     curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=channel)
#     curve.opts(
#         subcoordinate_y=True, color="black", line_width=1, tools=[hover],
#     )
#     channel_curves.append(curve)

# eeg = hv.Overlay(channel_curves, kdims="Channel").opts(
#     xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True,
# )

# y_positions = range(N_CHANNELS)
# yticks = [(i , ich) for i, ich in enumerate(channels)]

# z_data = zscore(data, axis=1)

# minimap = rasterize(hv.Image((time, y_positions , z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
# minimap = minimap.opts(
#     cmap="RdBu_r", xlabel='Time (s)', alpha=.5, yticks=[yticks[0], yticks[-1]],
#     height=150, responsive=True, default_tools=[], clim=(-z_data.std(), z_data.std())
# )

# RangeToolLink(
#     minimap, eeg, axes=["x", "y"],
#     boundsx=(None, 2), boundsy=(None, 6.5)
# )

# dashboard = (eeg + minimap).opts(merge_tools=False).cols(1)
# dashboard

### subcoords testing synthetic

In [ ]:
max_ch_disp = 20  # max channels to initially display
max_t_disp = 20 # max time in seconds to initially display

annotation = hv.VSpan(1, 1.5) # example annotation (start, end) time

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, tools=[hover])
    channel_curves.append(curve)

eeg_viewer = (hv.Overlay(channel_curves, kdims="Channel"))
eeg_viewer = eeg_viewer.opts(
    padding=0, xlabel="Time (s)", ylabel="Channel",
    show_legend=False, aspect=1.5, responsive=True,
    shared_axes=False, xlim=(time.min(), time.max()), backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (-.5, len(channels)-.5)})

# y_positions = range(len(channels))
y_positions, _ = zip(*yticks)
yticks = [(i, ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))

clim_mul = 1.2
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=100, responsive=True, default_tools=[], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# Create RangeToolLink between the minimap and the main EEG viewer 
# max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
RangeToolLink(minimap, eeg_viewer, axes=["x", "y"], boundsx=(None, 2), boundsy=(None, 6.5))

eeg_app = (eeg_viewer + minimap).opts(merge_tools=False).cols(1)

# eeg_app = pn.Column((eeg_viewer + minimap * annotations_overlay).cols(1), min_height=650)
# eeg_app

## Real data pipeline

In [ ]:
import mne

### Intake data

In [ ]:
# This dataset is 2.6 MB on disk
url = "https://physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf?download"
local_data_path = "../../data/"

# Will not download if already present at local_data_path
local_file_path = download_file(url, local_data_path)

In [ ]:
raw = mne.io.read_raw_edf(local_file_path, preload=True)
raw.info

In [ ]:
# preview the channel names, types, signal ranges, and uncompressed size
raw.describe()

### Gather the real timeseries annotations and clean up

In [ ]:
# get initial time of experiment
orig_time = raw.annotations.orig_time

# get annotations into pd df
annotations_df = raw.annotations.to_data_frame()

# Ensure the 'onset' column is in UTC timezone
annotations_df['onset'] = annotations_df['onset'].dt.tz_localize('UTC')

annotations_df['start'] = (annotations_df['onset'] - orig_time).dt.total_seconds()
annotations_df['end'] = annotations_df['start'] + annotations_df['duration']


unique_descriptions = annotations_df['description'].unique()
color_map = dict(zip(unique_descriptions, cc.glasbey[:len(unique_descriptions)]))
annotations_df['color'] = annotations_df['description'].map(color_map)

annotations_df.head()


### Clean channel names, set sensor positions, and reference data

In [ ]:
# clean up the channel names
raw.rename_channels(lambda s: s.strip("."));

In [ ]:
# optional: preview available montages that are shipped with MNE
# mne.channels.get_builtin_montages(descriptions=True)

In [ ]:
# optional: Let's use the standard 10-20
# montage = mne.channels.make_standard_montage("standard_1020")

In [ ]:
# optional: plot the assigned positions of our data channels
# raw.set_montage(montage, match_case=False)
# sphere=(0, 0.015, 0, 0.099) #manually adjust the y origin coord and radius a bit
# raw.plot_sensors(show_names=True, sphere=sphere);

In [ ]:
# re-reference EEG data to the average over all recording channels
raw.set_eeg_reference("average");

### Gather the data for plotting into simple numpy arrays

In [ ]:
time = raw.times
channels = raw.ch_names

# get the EEG data (for this data set, all channels are EEG anyways)
eeg_indices = mne.pick_types(raw.info, eeg=True)
data = raw.get_data(picks=eeg_indices, units={"eeg":"uV"})

### Visualize real data

In [ ]:
# max_ch_disp = 20  # max channels to initially display
# max_t_disp = 20 # max time in seconds to initially display

# spacing = 2.5  # Spacing between channels
# offset = np.std(data) * spacing

# # Create an overlay of VSpan annotations based on the annotations dataframe
# annotation_elements = [hv.VSpan(row['start'], row['end']).opts(fill_color=row['color'], alpha=0.1) 
#                        for _, row in annotations_df.iterrows()]
# annotations_overlay = hv.Overlay(annotation_elements)

# # Create a hv.Curve element per chan
# channel_curves = []
# max_data = data.max()
 
# hover = HoverTool(tooltips=[
#     ("Channel", "@channel"),
#     ("Time", "$x s"),
#     ("Amplitude", "@original_amplitude µV")])

# xwheel = WheelZoomTool(
#     zoom_together="none",
#     dimensions="width",
#     maintain_focus=False,
# )

# for i, channel_data in enumerate(data):
#     offset_data = channel_data + (i * offset)
#     max_data = max(offset_data.max(), max_data) # update max
#     ds = Dataset((time, offset_data, channel_data, channels[i]), ["Time", "Amplitude", "original_amplitude", "channel"])
#     channel_curves.append(
#         hv.Curve(ds, "Time", ["Amplitude", "original_amplitude", "channel"]).opts(
#             color="black", line_width=1,
#             tools=[hover, xwheel], shared_axes=False))

# yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# def set_maintain_focus(plot, element):
#     wheel_zoom = plot.state.select(type=WheelZoomTool)
#     if wheel_zoom:
#         wheel_zoom[0].maintain_focus = False

# # Create an overlay of curves
# eeg_viewer = (annotations_overlay * hv.Overlay(channel_curves, kdims="Channel"))
# eeg_viewer = eeg_viewer.opts(
#     padding=0, xlabel="Time (s)", ylabel="Channel",
#     yticks=yticks, show_legend=False, aspect=1.5, responsive=True,
#     shared_axes=False, xlim=(time.min(), time.max()), backend_opts={
#         # "x_range.bounds": (time.min(), time.max()),
#         "y_range.bounds": (data.min(), max_data)},
#     hooks=[set_maintain_focus])

# # Get the y positions of the yticks to use as yaxis of minimap image
# y_positions, _ = zip(*yticks)

# # Compute z-scores across time for each channel
# z_data = zscore(data, axis=1)

# # Generate the zscored image for the minimap using the y tiack positions from the eeg_viewer
# minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))

# # Style the minimap 
# clim_mul = 1.2
# minimap = minimap.opts(
#     cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
#     height=100, responsive=True, default_tools=[''], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))
    
# # Create RangeToolLink between the minimap and the main EEG viewer 
# max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
# RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"],
#               boundsx=(None, max_t_disp),
#               boundsy=(None, max_y_disp))

# eeg_app = pn.Column((eeg_viewer + minimap * annotations_overlay).cols(1), min_height=650)
# eeg_app

### Subcoords testing real

In [ ]:
max_ch_disp = 20  # max channels to initially display
max_t_disp = 20 # max time in seconds to initially display

spacing = 2.5  # Spacing between channels
offset = np.std(data) * spacing

# Create an overlay of VSpan annotations based on the annotations dataframe
# annotation_elements = [hv.VSpan(row['start'], row['end']).opts(fill_color=row['color'], alpha=0.1) 
#                        for _, row in annotations_df.iterrows()]
# annotations_overlay = hv.Overlay(annotation_elements)

annotation = hv.VSpan(19, 20).opts(color='yellow', alpha=.15) # example annotation (start, end) time

channel_curves = []

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

for channel, channel_data in zip(channels, data):
    ds = Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, subcoordinate_scale=3, tools=[hover])
    channel_curves.append(curve)

eeg_viewer = (annotation*hv.Overlay(channel_curves, kdims="Channel"))
eeg_viewer = eeg_viewer.opts(
    padding=0, xlabel="Time (s)", ylabel="Channel",
    show_legend=False, aspect=2, responsive=True,
    shared_axes=False, xlim=(time.min(), time.max()), backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (0, len(channels))})


y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))

clim_mul = 3
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=125, responsive=True, default_tools=[], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# Create RangeToolLink between the minimap and the main EEG viewer 
# max_y_disp = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
RangeToolLink(minimap, eeg_viewer, axes=["x", "y"], boundsx=(15, 25), boundsy=(17, 33))

eeg_app = (eeg_viewer + minimap*annotation).opts(merge_tools=False).cols(1)

# eeg_app = pn.Column((eeg_viewer + minimap * annotations_overlay).cols(1), min_height=650)
eeg_app = pn.Column((eeg_viewer + minimap * annotation).cols(1), min_height=650).servable(target='main', title='EEG Viewer with HoloViz and Bokeh')
eeg_app